In [1]:
    import abc, jsonschema, rdflib, collections, copy, pyld.jsonld as jsonld, ast, pathlib
    import strict_rfc3339, rfc3986, rfc3987, IPython

In [2]:
    pathlib.Path.text = pathlib.Path.read_text
    pathlib.Path.json = lambda x: __import__('json').loads(x.text())

In [3]:
    class Schema:
        @classmethod
        def schema(cls): return {**collections.ChainMap(*(getattr(object, '__annotations__', {}) or {} for object in cls.__mro__)), 'title': cls.__name__, }
        @classmethod
        def validate(cls, object, schema=None): jsonschema.validate(object, schema or cls.schema(), format_checker=jsonschema.FormatChecker())

In [4]:
    class JsonSchemaMeta(Schema, abc.ABCMeta):
        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = cls.validate(kwargs) or kwargs

    class JsonSchema(JsonSchemaMeta, **jsonschema.Draft7Validator.META_SCHEMA): 
        def __new__(cls, object=None, *args, **kwargs):
            object= super().__new__(cls, object, *args, **kwargs)
            return object

In [5]:
    class Pandas:
        def frame(self): return __import__('pandas').DataFrame(self)
        def series(self): return __import__('pandas').Series(self)        

In [24]:
    class Type(Schema, metaclass=JsonSchema): 
        __annotations__ = {}
        def __new__(cls, object=None, *args, **kwargs):
            schema = cls.schema()
            if object is None: object = copy.copy(schema.get('default'))
            object = super().__new__(cls, object, *args, **kwargs)
            return cls.validate(object) or object

        def __init_subclass__(cls, **kwargs):  cls.__annotations__ = type(cls).validate(kwargs) or kwargs

        @classmethod
        def object(type, object=None):
            for cls in (type.__subclasses__()):
                try: return cls.object(object)
                except BaseException as e: ...
            else: return type(object)
    class List(Pandas, Type, list, type='array', default=[]): ... 
    class Dict(Pandas, Type, dict, type='object', default={}): ...

    class Null(Type, type='null'): 
        def __new__(cls, object=None, *args, **kwargs): return cls.validate(cls, object)
    class Integer(Type, int, type='integer', default=''): ...
    class Number(Type, float, type='number', default=''): ...
    
    class String(Type, str, type='string', default=''): 
        def text(self): return str(self)
        def yaml(self): 
            try: from ruamel import yaml
            except: import yaml
            return Type.object(yaml.safe_load(self.text()))
        def json(self): 
            try: import ujson as json
            except: import json
            return json.loads(self.text())
        def path(self): return pathlib.Path(self)
    class Date(String, format='date'): ...
    class Datetime(String, format='date-time'): ...
    class Time(String, format='time'): ...
    class Color(String, format='color'): ...
    
    class Email(String, format='email'): ...
    class Uri(String, format='uri'):         
        def text(self, **object): return __import__('requests').get(self, **object).text
    
        
    class File(String): 
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().exists(): return 
            raise ValueError(F"{object} is not a file.")
    class Dir(File): 
        @classmethod
        def validate(cls, object, schema=None):
            if object.path().is_dir(): return 
            raise ValueError(F"{object} is not a file.")

In [21]:
    class StringTypes(ast.NodeTransformer):
        def visit_Str(self, node):
            next = ast.parse(F"""__import__('importlib').import_module('{__name__}').String.object('')""").body[0].value
            next.args = [node]
            return ast.copy_location(next, node)
        visit_JoinedStr = visit_Str

In [8]:
    def unload_ipython_extension(shell): shell.ast_transformers = [x for x in shell.ast_transformers if not isinstance(x, StringTypes)]
    def load_ipython_extension(shell): unload_ipython_extension(shell) or shell.ast_transformers.append(StringTypes())
    __name__ == '__main__' and load_ipython_extension(get_ipython())